In [99]:
import requests
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import musicbrainzngs

In [2]:
class MusicStream:
    def __init__(self, genero, año):
        self.genero = genero
        self.año = año
    
    def spotify(self, genero, año):
        client_id = "9ba747b4cd1a426c826e14373c7648a1"
        api_key = "4c67909da02740418998f8277a500767"

        auth_manager = SpotifyClientCredentials( client_id, api_key)
        sp = spotipy.Spotify(auth_manager=auth_manager)

        tracks ={"canciones":[], "artistas": [], "género": genero, "tipo": "track", "año lanzamiento": [], "año": año}
        for offset in range(0, 500, 50):
            canciones = sp.search(q= f"genre:{genero} year: {año}", type="track", limit=50, offset=offset)
            for track in canciones["tracks"]["items"]:
                if track not in tracks["canciones"]:
                    tracks["canciones"].append(track["name"])
                    tracks["artistas"].append(track["artists"][0]["name"])
                    try:
                        tracks["año lanzamiento"].append(track["album"]["release_date"].split("-")[0])
                    except:
                        tracks["año lanzamiento"].append("no data") 
                else: 
                    pass
        return tracks
    def musicbrainz(self, genero, año):
        tracks = self.spotify(genero, año)
        musicbrainzngs.set_useragent("MusicStream", "1.0", "psainzpc@hotmail.com")
        lista_artistas = list(set(tracks["artistas"]))  # Convertir el valor asociado a la clave 'artistas' en un set para limpiar los artistas repetidos
        artistas = {"artista": [], "pais de origen": [], "area de origen": [], "fecha inicio actividad": [], "fin de actividad": []} 
        for a in lista_artistas:
            result = musicbrainzngs.search_artists(artist=a)
            try:
                artistas["artista"].append(result["artist-list"][0]["name"])
            except:
                artistas["artista"].append("no data")
            try:
                artistas["pais de origen"].append(result["artist-list"][0]["area"]["name"])
            except:
                artistas["pais de origen"].append("no data")
            try:
                artistas["area de origen"].append(result["artist-list"][0]["begin-area"]["name"])
            except:
                artistas["area de origen"].append("no data") 
            try:
                artistas["fecha inicio actividad"].append(result["artist-list"][0]["life-span"]["begin"].split("-")[0])
            except:
                artistas["fecha inicio actividad"].append("no data")
            try:
                if result["artist-list"][0]["life-span"]["ended"] == False:
                    artistas["fin de actividad"].append("no activo")
                else:
                    artistas["fin de actividad"].append("activo")
            except:
                artistas["fin de actividad"].append("no data")
        return artistas
    def lastfm(self, genero, año):
        api_key= "cce24bdeb4a50d132f3b117a6dbf6dab"
        tracks = self.spotify(genero, año)
        lista_artistas = list(set(tracks["artistas"])) 
        artistas_lfm = {"Artista":[], "Biografia":[], "Listeners":[], "Playcount":[], "Artistas Similares":[]}
        for a in lista_artistas:
            params_info = {
                'method': 'artist.getinfo',
                'artist': a,
                'api_key': api_key,
                'format': 'json'
                }
            url = f"http://ws.audioscrobbler.com/2.0/"
            respuesta = requests.get(url, params=params_info)
            artista = respuesta.json()
            try:
                artistas_lfm['Artista'].append(artista["artist"]["name"])
            except:
                artistas_lfm["Artista"].append(None)
            try:
                artistas_lfm["Biografia"].append(artista["artist"]["bio"]["summary"])
            except:
                artistas_lfm["Biografia"].append(None)
            try:
                artistas_lfm["Listeners"].append(artista["artist"]["stats"]["listeners"])
            except:
                artistas_lfm["Listeners"].append(None)
            try:
                artistas_lfm["Playcount"].append(artista["artist"]["stats"]["playcount"])
            except:
                artistas_lfm["Playcount"].append(None)
            try:
                artistas_lfm["Artistas Similares"].append(artista["artist"]["similar"]["artist"][0]["name"])
            except:
                artistas_lfm["Artistas Similares"].append(None)
        return artistas_lfm

In [5]:
pop = MusicStream("pop", 2023)

In [139]:
pop.spotify("pop",2019)

{'canciones': ['Dreamland (feat. Years & Years) - 2023 Remaster',
  'The 2023 Gaming Rap Up',
  'Working in the Coal Mine - 2023 Remaster',
  'The 2023 Big Screen Rap Up',
  'This Is What It Feels Like - Armin van Buuren 2023 Remix',
  'Dreamland (feat. Years & Years) - 2023 Remaster',
  'New Year Mix 2023',
  '2023 (Friends)',
  'It Takes a Worried Man - 2023 Remaster',
  'Theme from Doctor Detroit - 2023 Remaster',
  'Puppet Boy - 2023 Remaster',
  'Gates of Steel - 2023 Remaster',
  'Girl U Want - 2023 Remaster',
  'Be Stiff (Stiff Version) - 2023 Remaster',
  'Beautiful World - 2023 Remaster',
  'Mongoloid (Warner Version) - 2023 Remaster',
  'Happy New Year 2023',
  'Happy New Year 2023',
  'Peek•A•Boo! - 2023 Remaster',
  'Whip It - 2023 Remaster',
  'Nel 2023 ( In the year 2525)',
  'Freedom of Choice - 2023 Remaster',
  'REFLEXION (ASOT 2023 Anthem) [Mixed]',
  'Jocko Homo (Warner Version) - 2023 Remaster',
  "That's Good - 2023 Remaster",
  'Big Mess - 2023 Remaster',
  'Secre

In [6]:
df_sp = pd.DataFrame(pop.spotify("pop",2023))
df_sp

,canciones,artistas,género,tipo,año lanzamiento,año
0,Dreamland (feat. Years & Years) - 2023 Remaster,Pet Shop Boys,pop,track,2020,2023
1,The 2023 Gaming Rap Up,NerdOut,pop,track,2023,2023
2,Working in the Coal Mine - 2023 Remaster,DEVO,pop,track,2023,2023
3,The 2023 Big Screen Rap Up,NerdOut,pop,track,2023,2023
4,This Is What It Feels Like - Armin van Buuren ...,Armin van Buuren,pop,track,2023,2023
...,...,...,...,...,...,...
495,IL0V3Y0U,Faouzia,pop,track,2024,2023
496,Shivers,Ed Sheeran,pop,track,2022,2023
497,Trailer Park,Jackson Dean,pop,track,2022,2023
498,Long Live,Florida Georgia Line,pop,track,2022,2023


In [141]:
pop.musicbrainz("pop",2023)

{'artista': ['Sailor',
  'Starship',
  'KC and the Sunshine Band',
  'DEVO',
  'twenty one pilots',
  'ARTY',
  'Iyaz',
  'Brando',
  'DKZ',
  'NerdOut',
  'Lukas Graham',
  'H. P. Lovecraft',
  'Dean Martin',
  'Bruno Mars',
  'The Monkees',
  'Forest Blakk',
  'Tritonal',
  '015B',
  'George R. R. Martin',
  'Mariette',
  'Fitz and the Tantrums',
  'Spandau Ballet',
  'CeeLo Green',
  'Omar Apollo',
  'Jenna Raine',
  'A.O.K.',
  'Alphaville',
  'Jason Mraz',
  'FORTEEN',
  'Kylie Minogue',
  'Bessie Smith',
  'Lady Gaga',
  'Panic! at the Disco',
  'Jess Glynne',
  'Danielle Bradbery',
  'Maxine Nightingale',
  'Gareth Emery',
  'Mase',
  '3LAU',
  'Shilpi Raj',
  'Saito Koji',
  'Armin van Buuren',
  'KSI',
  'The Beatles',
  'Charlie Puth',
  'Guddu Lal Yadav',
  'Abbey Cone',
  'Rudimental',
  'Tones and I',
  'Diana Ross',
  'a‐ha',
  'Soho Party',
  'Baani Sandhu',
  'Birdy',
  'Pat Boone',
  'David Guetta',
  'LIM',
  'Robyn',
  'Fateh Shergill',
  'Seven Lions',
  'Missy Elli

In [142]:
df_mb = pd.DataFrame(pop.musicbrainz("pop",2023))

In [144]:
df_mb

,artista,pais de origen,area de origen,fecha inicio actividad,fin de actividad
0,Sailor,United Kingdom,London,1973,activo
1,Starship,United States,no data,1984,activo
2,KC and the Sunshine Band,United States,Hialeah,1973,activo
3,DEVO,United States,Akron,1972,activo
4,twenty one pilots,United States,Columbus,2009,activo
...,...,...,...,...,...
128,Róisín Murphy,Ireland,Arklow,1973,activo
129,Sean Paul,Jamaica,Kingston,1973,activo
130,Bebe Rexha,United States,New York,1989,activo
131,Paramore,United States,Franklin,2004,activo


In [133]:
pop.lastfm("pop", 2023)

In [154]:
df_lfm = pd.DataFrame(pop.lastfm("pop", 2023))
df_lfm

,Artista,Biografia,Listeners,Playcount,Artistas Similares
0,Sailor,"British pop group founded in 1973, featuring P...",76625,425978,Racey
1,Starship,Starship is the name of more than one artist:\...,1070727,7985887,John Parr
2,KC & The Sunshine Band,KC and the Sunshine Band is an American musica...,446074,3378100,None
3,Devo,Devo are an American rock group formed in Akro...,1140949,22627206,Oingo Boingo
4,Twenty One Pilots,Twenty One Pilots (stylized in all lowercase o...,2484572,241975047,None
...,...,...,...,...,...
128,Róisín O,Dublin born singer/songwriter Róisín O has rec...,8078,33199,Hermitage Green
129,Sean Paul,Sean Paul Ryan Francis Henriques (born January...,2440997,33259002,Beenie Man
130,Bebe Rexha,"Bleta Rexha, known professionally as Bebe Rexh...",1194160,32062001,Rita Ora
131,Paramore,Paramore is an alternative rock band formed in...,4417245,384996195,Hayley Williams


In [204]:
def spotify(genero, año):
        client_id = "9ba747b4cd1a426c826e14373c7648a1"
        api_key = "4c67909da02740418998f8277a500767"

        auth_manager = SpotifyClientCredentials( client_id, api_key)
        sp = spotipy.Spotify(auth_manager=auth_manager)

        tracks ={"canciones":[], "artistas": [], "género": genero, "tipo": "track", "año lanzamiento": [], "año": año}
        for offset in range(0, 500, 50):
            canciones = sp.search(q= f"genre: ska, year: 2020", type="track", limit=50, offset=offset)
            for track in canciones["tracks"]["items"]:
                if track not in tracks["canciones"]:
                    tracks["canciones"].append(track["name"])
                    tracks["artistas"].append(track["artists"][0]["name"])
                    try:
                        tracks["año lanzamiento"].append(track["album"]["release_date"].split("-")[0])
                    except:
                        tracks["año lanzamiento"].append("no data") 
                else: 
                    pass
        return tracks

In [205]:
ska20 = spotify("ska", 2020)

In [206]:
listaska20 =list(set(ska20["artistas"]))

In [194]:
ska19 = spotify("ska", 2019)

In [196]:
listaska19 =list(set(ska19["artistas"]))

In [198]:
len(listaska19)

126

In [181]:
rock19 = spotify("rock", 2019)

In [151]:
len(rock19["canciones"])

500

In [182]:
listarock19 =list(set(rock19["artistas"]))

In [176]:
rock20 = spotify("rock", 2020)

In [177]:
listarock20 =list(set(rock20["artistas"]))

In [154]:
len(listarock19)

285

In [169]:
pop20 = spotify("pop", 2020)

In [170]:
len(pop20["canciones"])

500

In [171]:
listapop20 =list(set(pop20["artistas"]))

In [159]:
pop19 = spotify("pop", 2019)

In [161]:
listapop19 =list(set(pop19["artistas"]))

In [163]:
len(listapop19)

243

In [188]:
house19 = spotify("house", 2019)

In [189]:
len(house19["canciones"])

500

In [191]:
listahs19 =list(set(house19["artistas"]))


In [192]:
len(listahs19)

209

In [113]:
house20 = spotify("house", 2020)

In [112]:
listahs20 =list(set(house20["artistas"]))

In [207]:
def musicbrainz(genero, año):
        musicbrainzngs.set_useragent("MusicStream", "1.0", "psainzpc@hotmail.com")
        listaska20 # Convertir el valor asociado a la clave 'artistas' en un set para limpiar los artistas repetidos
        artistas = {"artista": [], "pais de origen": [], "area de origen": [], "fecha inicio actividad": [], "fin de actividad": []} 
        for a in listaska20:
            result = musicbrainzngs.search_artists(artist=a)
            try:
                artistas["artista"].append(result["artist-list"][0]["name"])
            except:
                artistas["artista"].append("no data")
            try:
                artistas["pais de origen"].append(result["artist-list"][0]["area"]["name"])
            except:
                artistas["pais de origen"].append("no data")
            try:
                artistas["area de origen"].append(result["artist-list"][0]["begin-area"]["name"])
            except:
                artistas["area de origen"].append("no data") 
            try:
                artistas["fecha inicio actividad"].append(result["artist-list"][0]["life-span"]["begin"].split("-")[0])
            except:
                artistas["fecha inicio actividad"].append("no data")
            try:
                if result["artist-list"][0]["life-span"]["ended"] == False:
                    artistas["fin de actividad"].append("no activo")
                else:
                    artistas["fin de actividad"].append("activo")
            except:
                artistas["fin de actividad"].append("no data")
        return artistas

In [208]:
df_mb_ska20 = pd.DataFrame(musicbrainz("ska", 2020))

In [209]:
df_mb_ska20.to_csv("ska20_mb.csv", index = False)

In [200]:
df_mb_ska19 = pd.DataFrame(musicbrainz("ska", 2019))

In [202]:
df_mb_ska19.to_csv("ska19_mb.csv", index = False)

In [184]:
df_mb_rock19 = pd.DataFrame(musicbrainz("rock", 2019))

In [185]:
df_mb_rock19.to_csv("rock19_mb.csv", index = False)

In [179]:
df_mb_rock20 = pd.DataFrame(musicbrainz("rock", 2020))

In [180]:
df_mb_rock20.to_csv("rock20_mb.csv", index = False)

In [173]:
df_mb_pop20 = pd.DataFrame(musicbrainz("pop", 2020))

In [174]:
df_mb_pop20.to_csv("pop20_mb.csv", index = False)

In [165]:
df_mb_pop19 = pd.DataFrame(musicbrainz("pop", 2019))

In [167]:
df_mb_pop19.to_csv("pop19_mb.csv", index = False)

In [118]:
df_mb_h19 = pd.DataFrame(musicbrainz("house", 2019))

In [120]:
df_mb_h19.to_csv("house19_mb.csv", index = False)

In [107]:
df_mb_h20 = pd.DataFrame(musicbrainz("house", 2020))

In [115]:
df_mb_h20.to_csv("house20_mb.csv", index = False)

In [233]:
def lastfm():
        api_key= "cce24bdeb4a50d132f3b117a6dbf6dab"
        listaska20
        artistas_lfm = {"Artista":[], "Biografia":[], "Listeners":[], "Playcount":[], "Artistas Similares":[]}
        for a in listaska20:
            params_info = {
                'method': 'artist.getinfo',
                'artist': a,
                'api_key': api_key,
                'format': 'json'
                }
            url = f"http://ws.audioscrobbler.com/2.0/"
            respuesta = requests.get(url, params=params_info)
            artista = respuesta.json()
            try:
                artistas_lfm['Artista'].append(artista["artist"]["name"])
            except:
                artistas_lfm["Artista"].append(None)
            try:
                artistas_lfm["Biografia"].append(artista["artist"]["bio"]["summary"])
            except:
                artistas_lfm["Biografia"].append(None)
            try:
                artistas_lfm["Listeners"].append(artista["artist"]["stats"]["listeners"])
            except:
                artistas_lfm["Listeners"].append(None)
            try:
                artistas_lfm["Playcount"].append(artista["artist"]["stats"]["playcount"])
            except:
                artistas_lfm["Playcount"].append(None)
            try:
                artistas_lfm["Artistas Similares"].append(artista["artist"]["similar"]["artist"][0]["name"])
            except:
                artistas_lfm["Artistas Similares"].append(None)
        return artistas_lfm

In [236]:
df_lfm_ska20 = pd.DataFrame(lastfm())

In [237]:
df_lfm_ska20.to_csv("ska20_lfm.csv", index = False)

In [234]:
df_lfm_ska19 = pd.DataFrame(lastfm())

In [235]:
df_lfm_ska19.to_csv("ska19_lfm.csv", index = False)

In [231]:
df_lfm_rock19 = pd.DataFrame(lastfm())

In [232]:
df_lfm_rock19.to_csv("rock19_lfm.csv", index = False)

In [228]:
df_lfm_rock20 = pd.DataFrame(lastfm())

In [229]:
df_lfm_rock20.to_csv("rock20_lfm.csv", index = False)

In [225]:
df_lfm_pop20 = pd.DataFrame(lastfm())

In [226]:
df_lfm_pop20.to_csv("pop20_lfm.csv", index = False)

In [222]:
df_lfm_pop19 = pd.DataFrame(lastfm())

In [223]:
df_lfm_pop19.to_csv("pop19_lfm.csv", index = False)

In [219]:
df_lfm_h19 = pd.DataFrame(lastfm())

In [220]:
df_lfm_h19.to_csv("house19_lfm.csv", index = False)

In [216]:
df_lfm_h20 = pd.DataFrame(lastfm())

In [217]:
df_lfm_h20.to_csv("house20_lfm.csv", index = False)